In [5]:
import tensorflow as tf 
from tensorflow import keras
import os 
import numpy as np 
import matplotlib.pyplot as plt
import segmentation_models as sm
from glob import glob


In [6]:
base_path = 'C:/Users/user/Desktop/Atopy Segmentation'
path_8 = os.path.join(base_path, 'Intersect_0.8')

In [12]:
x_train = glob(os.path.join(path_8, 'Atopy_Segment_Train', '*/*.jpg'))
y_train = glob(os.path.join(path_8, 'Atopy_Segment_Train', '*/*.png'))

x_test = glob(os.path.join(path_8, 'Atopy_Segment_Test', '*/*.jpg'))
y_test = glob(os.path.join(path_8, 'Atopy_Segment_Test', '*/*.png'))

x_val = glob(os.path.join(path_8, 'Atopy_Segment_Extra', '*/*.jpg'))
y_val = glob(os.path.join(path_8, 'Atopy_Segment_Extra', '*/*.png'))

In [13]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [14]:
# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)


In [15]:
# define model
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator
model.fit(
   x=x_train,
   y=y_train,
   batch_size=16,
   epochs=100,
   validation_data=(x_val, y_val),
)

KeyboardInterrupt: 